# What to do

- make the kernel from https://www.kaggle.com/wolfgangb33r/simple-talkingdata-prediction-xgboost/code

# Setup

## Libraries

In [1]:
import pandas as pd
import git
from pandas import DataFrame, Series
from pandas.api.types import CategoricalDtype

import os
import sys
repo = git.Repo('.', search_parent_directories=True)
module_path = repo.working_tree_dir
print(module_path + "\t<<<<<< get to project")
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import datetime as dt
from datetime import date
from boto.s3.connection import S3Connection
import tarfile

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


/home/ec2-user/kaggle/kaggle_talkingdata	<<<<<< get to project


In [2]:
#Importing the required libraries
import zipfile
import subprocess
import psutil
import base64
import zlib
import glob
import dask.dataframe as dd
from dask.delayed import delayed
import dask.bag as db
import calendar
import glob
import time
from sys import getsizeof
from boto.s3 import key
from datetime import datetime,timedelta
from boto.s3.key import Key

In [3]:
import numpy as np
import pandas as pd
import math
import time
import os.path

import xgboost as xgb

## Other notebooks

In [4]:
import user_defined_functions.notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())

In [5]:
import user_defined_functions.helper_functions as hp

## AWS

In [6]:
aws_connection = S3Connection(os.environ["AWS_ACCESS_KEY_ID"], os.environ["AWS_SECRET_ACCESS_KEY"])
#bucket = aws_connection.get_bucket('sshimizu-tokyo')

In [7]:
bucket = aws_connection.get_bucket('awsant-lalaport-data')

## Parameters

In [8]:
main_dir = "s3://awsant-lalaport-data/temp/kaggle/kaggle_talkingdata/"

In [9]:
input_file = main_dir + "data/train.csv.gz"

In [10]:
test_file = main_dir + "data/test.csv"

In [11]:
sample_submission_file = main_dir + "data/sample_submission.csv"

In [12]:
output_file = module_path + '/submit/xgb_wolfgangbeer_400milv3.csv'

- v1: test_model, 40,000 per chunk at 10 chunks
- v2: bigger model, 400,000 per chunk at 10 chunks
- v3: biggest model, 4,000,000 per chunk at 10 chunks (should fit all data)

# Main Code

In [13]:
start_time = time.time()
# create a xgboost model
model = xgb.XGBClassifier(max_depth=3, n_estimators=600, learning_rate=0.05)

## Test Read

In [14]:
df = pd.read_csv(input_file, nrows = 100)

In [15]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


## Read

### Training data

In [16]:
# already prepared for chunked learning in case that full training data
# set does not fit into memory
# fit the model
chunksize = 4000000
chunk_idx = 0 
chunk_max = 10
for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Extracting new features
    chunk['hour'] = pd.to_datetime(chunk.click_time).dt.hour.astype('uint8')
    chunk['day'] = pd.to_datetime(chunk.click_time).dt.day.astype('uint8')
    train_X = chunk.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'hour', 'day'])
    if chunk_idx > 0: # not load in first run
        model.fit(train_X, chunk['is_attributed'], xgb_model=model)
    else:
        model.fit(train_X, chunk['is_attributed'])
    print("Processed chunk %d" % chunk_idx)
    chunk_idx = chunk_idx + 1
    if chunk_idx >= chunk_max:
        break

Processed chunk 0
Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8


KeyboardInterrupt: 

### Testing data

In [17]:
# read test data set
test = pd.read_csv(test_file)
test['hour'] = pd.to_datetime(test.click_time).dt.hour.astype('uint8')
test['day'] = pd.to_datetime(test.click_time).dt.day.astype('uint8')

test_X = test.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'hour', 'day'])
#start_time = print_duration (start_time, "Finished training, start prediction")   
# predict the propabilities for binary classes    

In [18]:
test_X.shape

(18790469, 7)

In [19]:
pred = model.predict_proba(test_X)

### Submission

In [20]:
submission = pd.read_csv(sample_submission_file)
submission['is_attributed'] = pred[:,1]
submission.to_csv(output_file, index=False)

In [21]:
submission.head()

,click_id,is_attributed
0,0,0.000359
1,1,0.000483
2,2,0.000419
3,3,0.000112
4,4,0.000293
